In [1]:
from matplotlib import pyplot as plt
import cv2
import time
import nibabel as nib
import os
import numpy as np
from shutil import rmtree
from threading import Thread

In [2]:
image_folder = './data/MICCAI_BraTS2020_TrainingData/'
all_images_folder = [image_folder + f + '/' for f in os.listdir(image_folder)][:-2]

In [3]:
training_folder = './gan_training_data/'

In [4]:
rmtree(training_folder)
os.mkdir(training_folder)
os.mkdir(training_folder + 'trainA')
os.mkdir(training_folder + 'trainB')
os.mkdir(training_folder + 'valA')
os.mkdir(training_folder + 'valB')
os.mkdir(training_folder + 'testA')
os.mkdir(training_folder + 'testB')

In [5]:
image_file_format = '{}{}_{}.nii.gz'

def get_images(file):
    img = nib.load(file)
    data = img.get_fdata()
    maxx = data.max()
    data = data/maxx
    
    return data, data.shape[-1]

In [6]:
def get_save_folder():
    rand = np.random.random()
    
    if rand<0.15:
        return 'val'
    elif rand<0.3:
        return 'test'
    else:
        return 'train'

In [7]:
def save_image_data_train(image_folder):
    file_path_t1 = image_file_format.format(image_folder, image_folder.split('/')[-2], 't1')
    file_path_t2 = image_file_format.format(image_folder, image_folder.split('/')[-2], 't1ce')
    
    t1_img, _ = get_images(file_path_t1)
    t2_img, _ = get_images(file_path_t2)
    
    save_folder = get_save_folder()
    file_name = image_folder.split('/')[-2].split('_')[-1]
    for i in range(27, 127):
        cv2.imwrite(
            training_folder + save_folder+ 'A/' + file_name + '_' + str(i) + '.jpg', 
            (t1_img[:, :, i] * 255).astype('int')
        )
        
        cv2.imwrite(
            training_folder + save_folder+ 'B/' + file_name + '_' + str(i) + '.jpg', 
            (t2_img[:, :, i] * 255).astype('int')
        )

In [8]:
processing_threads = []
max_threads = 10

for idx, img_folder in enumerate(all_images_folder):
    print('Processing at index {}'.format(idx), end='\r')
    processing_threads.append(Thread(target=save_image_data_train, args=[img_folder]))
    processing_threads[-1].start()
    
    if len(processing_threads) == max_threads:
        while all([t.is_alive() for t in processing_threads]):
            time.sleep(0.3)
        
        for t in reversed(processing_threads):
            if not t.is_alive():
                processing_threads.remove(t)

In [25]:
save_image_data_train(all_images_folder[0])

In [24]:
all_images_folder[0]

'./data/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/'